In [1]:
!pip install langchain_groq langchain_core langchain_community

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is lord Ram?")
print(result.content)

Lord Rama is a major deity in Hinduism and is considered the seventh avatar (incarnation) of the god Vishnu. He is the main character in the ancient Indian epic, the Ramayana, which is one of the most revered and influential texts in Hinduism.

According to Hindu mythology, Lord Rama was born as a prince in the kingdom of Ayodhya, to King Dasharatha and Queen Kausalya. He is known for his exceptional virtues, such as truthfulness, courage, and devotion to his family and duty.

The story of Lord Rama is as follows:

* He was exiled to the forest for 14 years by his stepmother, Queen Kaikeyi, who wanted her own son, Bharata, to become the king.
* During his exile, Rama's wife, Sita, was abducted by the demon king Ravana, who took her to his kingdom in Lanka (modern-day Sri Lanka).
* Rama, along with his brother Lakshmana and the monkey god Hanuman, embarked on a journey to rescue Sita and defeat Ravana.
* After a long and arduous battle, Rama defeated Ravana and rescued Sita, returning t

In [3]:
!pip install pypdf

In [4]:
!pip install chromadb

In [5]:
!pip install sentence_transformers

In [6]:
!pip install langchain-classic

In [9]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_classic.chains import RetrievalQA
from langchain_classic.prompts import PromptTemplate
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
import os
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/Data", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = '/content/chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()


Intializing Chatbot.........


/tmp/ipython-input-2745606708.py:56: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)



Human: hi


/tmp/ipython-input-2745606708.py:64: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  response = qa_chain.run(query)


Chatbot: Hello. It's nice to meet you. I see you've been reading about mental health, specifically about how it's defined and the importance of considering an individual's social and physical environment. It's a fascinating topic, and I'm happy to discuss it with you.

The definitions you provided from Rowling et al. (2002) and the World Health Organization (WHO) highlight the importance of having a sense of purpose, quality relationships, self-respect, and mastery in achieving good mental health. However, as you noted, these definitions can be incomplete, as they don't fully account for the impact of social and physical environments on an individual's mental wellbeing.

I think it's great that you're considering the complexities of mental health and how it's influenced by various factors. It's essential to recognize that mental health is not just an individual issue, but also a collective one, shaped by our relationships, communities, and environments.

What are your thoughts on this 

In [10]:
!pip install gradio
import gradio as gr

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_classic.chains import RetrievalQA
from langchain_classic.prompts import PromptTemplate
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
import os
import gradio as gr

def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/data/", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = '/content/chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


print("Intializing Chatbot.........")
llm = initialize_llm()

db_path = "/content/chroma_db"

if not os.path.exists(db_path):
  vector_db  = create_vector_db()
else:
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(user_input, history = []):
  if not user_input.strip():
    return "Please provide a valid input", history
  response = qa_chain.run(user_input)
  history.append((user_input, response))
  return "", history

import gradio as gr

# 1. Logic to connect your Embeddings/QA Chain to the UI
def chatbot_response(message, history):
    # Ensure there is actual text to process
    if not message.strip():
        return "Please provide a valid input."

    try:
        # This uses your HuggingFaceBgeEmbeddings-backed chain
        # Note: If using newer LangChain, use qa_chain.invoke({"query": message})
        response = qa_chain.run(message)
        return response
    except Exception as e:
        return f"I'm sorry, I'm having a little trouble thinking right now. Error: {str(e)}"

# 2. Stable UI Setup
# We keep the theme here because launch(theme=...) is for a future version
with gr.Blocks(theme=gr.themes.Soft(primary_hue="emerald")) as app:
    gr.Markdown("# 🧠 Mental Health Chatbot 🤖")
    gr.Markdown("### A compassionate space for your well-being.")

    # gr.ChatInterface handles the 'history' internally
    gr.ChatInterface(
        fn=chatbot_response,
        type="messages"
    )

    gr.Markdown("---")
    gr.Markdown("> **Disclaimer:** This chatbot provides general support. For urgent issues, please contact a licensed professional or a local crisis hotline.")

# 3. Launch with explicit sharing for your notebook environment
app.launch(share=True, debug=True)

Intializing Chatbot.........


/tmp/ipython-input-898403686.py:85: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(primary_hue="emerald")) as app:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1aeb9b6e1e7dfe0c14.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
